In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
from matplotlib.ticker import MaxNLocator
import numpy as np
from datetime import datetime

dataset_path = '../data/raw/export_df.parquet'

In [ ]:
data = pd.read_parquet(dataset_path)
data.head()

df_copy = data.copy()

Opis kolona:

- datum_dolaska - datum kada bi gost trebao stići u hotel
- datum_kreiranja_rezervacije - datum kada je rezervacija kreirana
- datum_odjave - datum kada je gost napustio hotel
- datum_otkazivanja_rezervacije - datum kada je rezervacija otkazana
- broj_odraslih_gostiju - broj odraslih koji će boraviti u rezerviranim sobama
- broj_djece_gostiju - broj djece koji će boraviti u rezerviranim sobama
- zemlja_gosta - zemlja iz koje gost dolazi. Kategorije su zapisane u ISO 3155–3:2013 formatu
- kanal_prodaje_id - id kanala prodaje preko kojeg je rezervacija napravljena (primjeri: direktna rezervacija, agencija, online)
- tip_sobe_id - id tipa sobe koju je gost rezervirao
- cijena_nocenja - cijena jednog noćenja za pojedinu rezeraciju
- status_rezervacije - status rezervacije (Check-Out, Canceled, No-Show)
- rezervacija_id - id rezervacije
- gost_id - id gosta

In [ ]:
hotel_zero = data[data['hotel_id'] == 0]
hotel_one = data[data['hotel_id'] == 1]

assert (len(hotel_zero)+ len(hotel_one)) == len(data)

In [ ]:
print(f"Prvi hotel ima {len(hotel_zero)/len(data)*100} % svih zapisa ")
print(f"Drugi hotel ima {len(hotel_one)/len(data)*100} % svih zapisa ")



In [ ]:
print(hotel_zero['zemlja_gosta'].value_counts()[0:25])

In [ ]:
print(hotel_one['zemlja_gosta'].value_counts()[0:25])

In [ ]:
df_copy['datum_dolaska'] = pd.to_datetime(df_copy['datum_dolaska'])


df_copy['duljina_boravka'] = df_copy['datum_odjave'] - df_copy['datum_dolaska']
df_copy['duljina_boravka'] = df_copy['duljina_boravka'].dt.days


In [ ]:
min_date = df_copy['datum_dolaska'].min()
max_date = df_copy['datum_dolaska'].max()

print(f"Svi zapisi su u razdoblju od {min_date} do {max_date}")

### Vizuali

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_copy['is_cancellation'] = df_copy['datum_otkazivanja_rezervacije'].notna()

grouped_hotel = df_copy.groupby('hotel_id').agg({'is_cancellation': ['sum', 'count']})
grouped_hotel.columns = ['cancellations', 'total_bookings']
grouped_hotel['cancellation_percentage'] = (grouped_hotel['cancellations'] / grouped_hotel['total_bookings']) * 100

grouped_nationality = df_copy.groupby('zemlja_gosta').agg({'is_cancellation': ['sum', 'count']})
grouped_nationality.columns = ['cancellations', 'total_bookings']
grouped_nationality['cancellation_percentage'] = (grouped_nationality['cancellations'] / grouped_nationality['total_bookings']) * 100

grouped_channel = df_copy.groupby('kanal_prodaje_id').agg({'is_cancellation': ['sum', 'count']})
grouped_channel.columns = ['cancellations', 'total_bookings']
grouped_channel['cancellation_percentage'] = (grouped_channel['cancellations'] / grouped_channel['total_bookings']) * 100

In [ ]:
print(grouped_nationality['cancellation_percentage'][0:5])


In [ ]:
print(grouped_nationality['cancellations'][0:5])

In [ ]:
mean_price_hotel_zero, mean_price_hotel_one  =np.mean(hotel_zero['cijena_nocenja']), np.mean(hotel_one['cijena_nocenja'])
std_price_hotel_zero, std_price_hotel_one  =np.std(hotel_zero['cijena_nocenja']), np.std(hotel_one['cijena_nocenja'])


In [ ]:
q11 = np.percentile(hotel_zero['cijena_nocenja'], 25)
q31 = np.percentile(hotel_zero['cijena_nocenja'], 75)
iqr1 = q31 - q11
threshold1 = 1.5 * iqr1
outliers1 = np.where((hotel_zero['cijena_nocenja'] < q11 - threshold1) | (hotel_zero['cijena_nocenja'] > q31 + threshold1))
 
print(len(outliers1[0])/len(hotel_zero['cijena_nocenja'])*100)


In [ ]:
q12 = np.percentile(hotel_one['cijena_nocenja'], 25)
q32 = np.percentile(hotel_one['cijena_nocenja'], 75)
iqr2 = q31 - q12
threshold2 = 1.5 * iqr2
outliers2 = np.where((hotel_one['cijena_nocenja'] < q12 - threshold2) | (hotel_one['cijena_nocenja'] > q32 + threshold2))
 
print(len(outliers2[0])/len(hotel_one['cijena_nocenja'])*100)


In [ ]:
dict = plt.boxplot([hotel_zero['cijena_nocenja'], hotel_one['cijena_nocenja']], showfliers=False,showmeans=True)

### Datumi gdje je datum otkazivanja nakon datuma dolaska

In [ ]:
krivi_datumi = df_copy[df_copy['datum_otkazivanja_rezervacije'] > df_copy['datum_dolaska']]
print(f"Ukupno {len(krivi_datumi)} redaka s neispravnim datumima")
print(krivi_datumi.head(3))

#### Obrađivanje stupca datum dolaska 

In [ ]:
def process_arrival_date(df, years=None):
    df['datum_dolaska'] = pd.to_datetime(df['datum_dolaska'], errors='coerce')

    df['year'] = df['datum_dolaska'].dt.year
    df['month'] = df['datum_dolaska'].dt.month
    df['week'] = df['datum_dolaska'].dt.isocalendar().week.astype(float)
    df['day'] = df['datum_dolaska'].dt.day
    df['dayofweek'] = df['datum_dolaska'].dt.dayofweek
    df['quarter'] = df['datum_dolaska'].dt.quarter
    df['dayofyear'] = df['datum_dolaska'].dt.dayofyear

    if years is not None:
        if not isinstance(years, list):
            years = [years]  
        df = df[df['year'].isin(years)]

    return df


In [ ]:
df = process_arrival_date(df_copy)

df['booking_status'] = df['datum_otkazivanja_rezervacije'].notna().astype(int)

dolasci_po_hotelu = df.groupby(['datum_dolaska', 'hotel_id']).size().rename('booking_count').reset_index()

fig, ax = plt.subplots(2, 1, figsize=(16, 12))

sns.lineplot(data=dolasci_po_hotelu,
             x='datum_dolaska',
             y='booking_count',
             hue='hotel_id',
             ax=ax[0]
             )

ax[0].set_title('Broj dolaska po danu po hotelu')

otkazivanja_po_hotelu = df.groupby(['datum_dolaska', 'hotel_id'])['booking_status'].mean().reset_index()

sns.lineplot(data=otkazivanja_po_hotelu,
             x='datum_dolaska',
             y='booking_status',
             hue='hotel_id',
             ax=ax[1]
      )

ax[1].set_title('Prosječan broj otkazivanja po danu po hotelu')

plt.tight_layout()
plt.show()

### Isti vizual samo za 2015


In [ ]:
df = process_arrival_date(df_copy,years=[2015])

df['booking_status'] = df['datum_otkazivanja_rezervacije'].notna().astype(int)

dolasci_po_hotelu = df.groupby(['datum_dolaska', 'hotel_id']).size().rename('booking_count').reset_index()

fig, ax = plt.subplots(2, 1, figsize=(16, 12))

sns.lineplot(data=dolasci_po_hotelu,
             x='datum_dolaska',
             y='booking_count',
             hue='hotel_id',
             ax=ax[0]
             )

ax[0].set_title('Broj dolaska po danu po hotelu 2015')

otkazivanja_po_hotelu = df.groupby(['datum_dolaska', 'hotel_id'])['booking_status'].mean().reset_index()

sns.lineplot(data=otkazivanja_po_hotelu,
             x='datum_dolaska',
             y='booking_status',
             hue='hotel_id',
             ax=ax[1]
      )

ax[1].set_title('Prosječan broj otkazivanja po danu po hotelu 2015')

plt.tight_layout()
plt.show()

### Isti vizuali samo za 2016


In [ ]:
df = process_arrival_date(df_copy,years=[2016])

df['booking_status'] = df['datum_otkazivanja_rezervacije'].notna().astype(int)

dolasci_po_hotelu = df.groupby(['datum_dolaska', 'hotel_id']).size().rename('booking_count').reset_index()

fig, ax = plt.subplots(2, 1, figsize=(16, 12))

sns.lineplot(data=dolasci_po_hotelu,
             x='datum_dolaska',
             y='booking_count',
             hue='hotel_id',
             ax=ax[0]
             )

ax[0].set_title('Broj dolaska po danu po hotelu 2016')

otkazivanja_po_hotelu = df.groupby(['datum_dolaska', 'hotel_id'])['booking_status'].mean().reset_index()

sns.lineplot(data=otkazivanja_po_hotelu,
             x='datum_dolaska',
             y='booking_status',
             hue='hotel_id',
             ax=ax[1]
      )

ax[1].set_title('Prosječan broj otkazivanja po danu po hotelu 2016')

plt.tight_layout()
plt.show()

### Isti vizuali samo za 2017


In [ ]:
df = process_arrival_date(df_copy,years=[2017])

df['booking_status'] = df['datum_otkazivanja_rezervacije'].notna().astype(int)

dolasci_po_hotelu = df.groupby(['datum_dolaska', 'hotel_id']).size().rename('booking_count').reset_index()

fig, ax = plt.subplots(2, 1, figsize=(16, 12))

sns.lineplot(data=dolasci_po_hotelu,
             x='datum_dolaska',
             y='booking_count',
             hue='hotel_id',
             ax=ax[0]
             )

ax[0].set_title('Broj dolaska po danu po hotelu 2017')

otkazivanja_po_hotelu = df.groupby(['datum_dolaska', 'hotel_id'])['booking_status'].mean().reset_index()

sns.lineplot(data=otkazivanja_po_hotelu,
             x='datum_dolaska',
             y='booking_status',
             hue='hotel_id',
             ax=ax[1]
      )

ax[1].set_title('Prosječan broj otkazivanja po danu po hotelu 2017')

plt.tight_layout()
plt.show()

In [ ]:
feature_dates = ['day', 'week', 'year', 'quarter']

n_cols = 2
n_rows = math.ceil(len(feature_dates) / n_cols)
fig, ax = plt.subplots(n_rows, n_cols, figsize=(16, n_rows * 5))
ax = ax.flatten()

for i, feature in enumerate(feature_dates):
    sns.countplot(data=df_copy, x=feature, ax=ax[i], color='skyblue')
    ax[i].set_title(f'{feature.capitalize()} Distribution')
    ax[i].set_xlabel(feature.capitalize())
    ax[i].set_ylabel('Count')

    xticks = ax[i].get_xticklabels()
    xticks = [int(float(x.get_text())) for x in xticks if x.get_text()]
    ax[i].set_xticklabels(xticks, rotation=45)

    if len(ax[i].get_xticklabels()) > 10:
        ax[i].xaxis.set_major_locator(MaxNLocator(10))

for j in range(i + 1, len(ax)):
    ax[j].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
df_copy['duljina_boravka'] = df_copy['datum_odjave'] - df_copy['datum_dolaska']
df_copy['duljina_boravka'] = df_copy['duljina_boravka'].dt.days

In [ ]:
nationality_counts = df_copy['zemlja_gosta'].value_counts()

top_nationalities = nationality_counts.head(8)
top_nationalities['Others'] = nationality_counts[8:].sum()

plt.figure(figsize=(8, 8))
top_nationalities.plot.pie(autopct='%1.1f%%')
plt.title('Distibucija dolaženja po nacionalnosti gostiju')
plt.ylabel('') 


df_copy['nacionalnost'] = df_copy['zemlja_gosta'].apply(lambda x: x if x in nationality_counts.head(8).index.tolist() else 'Others')

plt.figure(figsize=(12, 8))
sns.boxplot(x='nacionalnost', y='duljina_boravka', data=df_copy, palette='Set3')
plt.title('Distribucija duljine boravka po nacionalnosti gostiju')
plt.xlabel('Nacionalnost')
plt.ylabel('Duljina boravka')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability

plt.show()
